# 해결해야 하는 문제

1. 현재 기본 리스트 한곳에 self.book_list_url 로 접근. input으로 여러 book_list url을 받았을 때, 똑같이 동작하는지 확인필요
2. good reads의 서로 다른 book list가 있을 때 중복 문제
---
만약 1,2 가 복잡하다면 그냥 지금 book_list_url 한 곳 (책 7만권)에 대해서만 진행
---
3. 종종 review 수집을 마치고 다음 책으로 넘어가지 못하고 뻗음 (무한로딩)
4. english 필터 걸고 리뷰 수집이 어려움 (정 안되면 그냥 필터 없이 모으고 데이터 전처리 때 걸러도 될지도)
5. 책 한권에 대해 리뷰를 아주 많이 ( num_reviews = 10000) 수집했을 때, 밴 문제 등
6. 책 100권 넘어가면 scrap list 구조 재편 필요 - 사이트에 100개까지라 다음 페이지 넘겨야 함.
---
7. 크롬드라이버 path 내 로컬에선 필요해서 설정했는데, 설정 안해도 된다면 수정
8. 혹시 또 모르는 문제가 있다면 공유해주시면 돕겠습니당

In [2]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time
import os
import pandas as pd
import json

class BookInfoCrawler:
    def __init__(self, output_dir: str, webdriver_path:str, num_books:int, num_reviews:int):

        """_summary_


        Args:
            output_dir (str): 결과 파일을 저장할 디렉토리
            webdriver_path (str): 크롬드라이버 경로
            num_books (int): 책 리스트에서 추출할 책 갯수
            num_reviews (int): 책 한권당 추출할 리뷰 갯수
        """


        # 출력 디렉토리
        self.output_dir = output_dir

        # 페이지에서 추출할 책 갯수

        # scrap list에서 총 추출할 책 갯수
        self.num_books = num_books

        # scrap reviews에서 책 한권당 추출할 리뷰 갯수
        self.num_reviews = num_reviews
        
        
        self.driver = None

        # 책 정보를 담는 임시리스트
        self.book_info: dict = {}


        # 최종 결과 저장용 변수 (필요 시 사용)
        self.output = None


        # ChromeDriver 경로 (실제 파일 위치로 수정)
        self.webdriver_path = webdriver_path





        # 로그인 오버레이 닫기 버튼의 CSS 셀렉터 (오버레이 닫기 버튼)
        self.login_overlay_dismiss_selector = "body > div.Overlay.Overlay--floating > div > div.Overlay__header > div > div > button"
        
        # 언어 필터 관련 xpath
        self.launguage_filter = {
            "filter": "//*[@id='ReviewsSection']/div[5]/div[2]/div[1]/div[2]/div/button",
            "en_label": "/html/body/div[3]/div/div[2]/span/div/div[6]/div[2]/label",
            "apply": "/html/body/div[3]/div/div[3]/div[2]/button",
            "text" : "/html/body/div[3]/div/div[2]/span/div/div[6]/div[2]/label/text()"
        }



        # scrap list -------------------------------------------------------------
        """
        _summary_
        scrap_list()에서
        1. 책 리스트 페이지에서 첫 번째 책 링크로 이동해 scrap_book() 호출
        2. scrap_book()에서 책 정보 추출, scrap_reviews() 호출하여 리뷰 수집
        3. 해당 책 정보 self.output에 추가
        4. 다음 책 링크로 이동하여 반복
        """

        # [scrap_list] 관련 변수: 책 리스트 페이지 URL 및 책 링크 요소 XPath 
        self.book_list_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1&ref=ls_pl_car_0"
        self.book_link_xpath = '//*[@id="all_votes"]/table/tbody/tr[1]/td[3]/a'

        # 책 리스트 페이지 관련 변수:
        self.book_list_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1&ref=ls_pl_car_0"
        # 기본적으로 한 페이지에 100권의 책이 있다고 가정

        self.book_link_xpath = "//*[@id='all_votes']/table/tbody/tr[1]/td[3]/a"  # 기본 템플릿(행 번호는 개별 처리)




        # scrap book에서 추출하는 xpath -------------------------------------------------------------
        """
        _summary_
        1. scrap_list()에서 scrap_book()을 호출
        2. self.xpath의 책 관련 정보 수집
        3. scrap_reviews() 호출
        """

        # scrap book 관련 변수
        self.book_detail_path = '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/div/button'
        self.xpath = {
            'book_name': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[1]/div[1]/h1',
            'book_image': '//*[@id="__next"]/div[2]/main/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/img',
            'author': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[1]/h3/div/span[1]/a/span',
            'total_star': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[2]/a/div[1]/div',
            'published': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'pages': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[1]/span/div/p[2]',
            'description': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[4]/div/div[1]/div/div/span',
            'ISBN': '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[6]/div/span[2]/div[1]/span/div/dl/div[3]/dd/div/div[1]'
        }
        


        # scrap reviews 에서 추출하는 xpath--------------------------------------------------------------
        """_summary_
        scrap_book() 에서
        1. view more reviews 버튼 클릭
        2. 리뷰 filter를 english로 제한 시도 ( 잘 안됨 )
        3. 리뷰 30개 스크롤, showmorereviews 눌러 30개 추가 로드... 반복
        
        """
        # [scrap_reviews] 관련 변수 

        # [scrap_book] 내 'view more reviews' 버튼 XPath 
        self.view_more_reviews_button_xpath = '//*[@id="ReviewsSection"]/div[6]/div[4]/a'
        

        # filter button----------- 리뷰 언어 설정 관련 xpath
        self.filter_button_xpath = '//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[4]/div[1]/div[2]/div/button'

        # 영어 선택 관련 XPath (수정된 값)
        self.english_label_xpath = '/html/body/div[3]/div/div[2]/span/div/div[6]/div[3]/label/span'
        # Apply 버튼의 xpath
        self.apply_button_xpath = '/html/body/div[3]/div/div[3]/div[2]/button'
        #-----------


        # show more reviews 버튼 xpath
        self.show_more_button_xpath = "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[5]/div/button"
        

        

        # 리뷰 내부 요소 xpath 템플릿 (INDEX를 실제 리뷰 컨테이너 번호로 대체)
        self.review_xpaths = {
            "reviewer": "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/div/div/section[2]/span[1]/div/a",
            "date":     "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/span/a",
            "rating":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[1]/div/span",
            "review":   "//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]/article/section/section[2]/section/div/div[1]/span"
        }
        

        # ...
    
    def start_browser(self):
        """
        Selenium WebDriver(Chrome) 객체를 생성하여 브라우저를 시작합니다.
        """
        options = Options()
        # 필요에 따라 headless 모드 사용 가능
        # options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/91.0.4472.124 Safari/537.36"
        )
        options.add_argument("--lang=ko_KR")
        options.add_argument("--charset=utf-8")
        
        self.driver = webdriver.Chrome(
            service=Service(self.webdriver_path),
            options=options
        )
    

    def apply_language_filter(self):
        """
        언어 필터 버튼을 눌러 오버레이를 연 후,
        label 텍스트가 "English"로 시작하는 언어 항목을 선택하고,
        Apply 버튼을 클릭하여 언어 필터를 적용합니다.
        """
        try:
            # 1. filter 버튼 클릭
            filter_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.launguage_filter['filter']))
            )
            filter_button.click()
            time.sleep(1)  # 오버레이 로딩 대기

            # 2. "English"로 시작하는 label을 선택 (오버레이 내)
            # 아래 XPath는 /html/body/div[3]/div/div[2]/span/div/div[6] 내의 각 언어 컨테이너에서
            # label 텍스트가 "English"로 시작하는 div를 찾아 그 안의 label 요소를 선택합니다.
            en_label = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "/html/body/div[3]/div/div[2]/span/div/div[6]/div[label[starts-with(normalize-space(text()), 'English')]]/label")
                )
            )
            en_label.click()
            time.sleep(1)  # 선택 후 잠시 대기

            # 3. apply 버튼 클릭 (오버레이 내)
            apply_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.launguage_filter['apply']))
            )
            apply_button.click()
            print("언어 필터 적용 완료")
        except Exception as e:
            print("언어 필터 적용 중 오류 발생:", e)



    def check_and_dismiss_overlay(self, timeout=5):
        """
        로그인 창 오버레이 해제용 함수
        주어진 timeout(초) 동안 로그인 오버레이 닫기 버튼이 나타나면 클릭하여 해제합니다.
        오버레이가 없으면 그냥 넘어갑니다.
        """
        end_time = time.time() + timeout
        dismissed = False
        while time.time() < end_time:
            try:
                dismiss_button = self.driver.find_element(By.CSS_SELECTOR, self.login_overlay_dismiss_selector)
                if dismiss_button.is_displayed() and dismiss_button.is_enabled():
                    dismiss_button.click()
                    print("로그인 오버레이가 해제되었습니다.")
                    dismissed = True
                    break
            except Exception:
                break
            time.sleep(0.5)
        return dismissed

    def scrap_book(self):
        """
        1. 도서 상세 페이지에서 책 정보를 추출합니다.
        2. 'view more reviews' 버튼을 클릭하기 전에 오버레이를 확인 및 제거합니다.
        3. 'view more reviews' 버튼을 클릭하여 리뷰 페이지로 이동합니다.
        4. scrap_reviews()를 호출하여 리뷰(영어 필터 적용) 데이터를 수집합니다.
        5. 책 정보와 리뷰를 포함한 딕셔너리를 반환합니다.
        """
        # book detail 누르기
        try:
            book_detail_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.book_detail_path))
            )
            WebDriverWait(self.driver, 10).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.Overlay.Overlay--floating"))
            )
            book_detail_button.click()
            time.sleep(2)
        except Exception as e:
            print("book details 버튼 클릭 중 오류 발생:", e)

        
        self.apply_language_filter()


        # 1. 책 정보 추출
        self.book_info = {}
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, self.xpath['book_name']))
            )
        except Exception as e:
            print("책 정보 로딩 중 오류 발생:", e)
            
        for key, xpath_val in self.xpath.items():
            try:
                element = self.driver.find_element(By.XPATH, xpath_val)
                if key == 'book_image':
                    self.book_info[key] = element.get_attribute("src")
                else:
                    self.book_info[key] = element.text.strip()
            except Exception as e:
                print(f"{key} 정보 추출 중 오류 발생: {e}")
                self.book_info[key] = None

        # genre 정보 추가
        genres = []
        index = 2  # span 인덱스는 2부터 시작

        while True:
            try:
                genre_xpath = f'//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[2]/div[2]/div[5]/ul/span[1]/span[{index}]/a/span'
                genre_element = self.driver.find_element(By.XPATH, genre_xpath)
                genre_text = genre_element.text.strip()
                genres.append(genre_text)
                index += 1  # 다음 하위 컨테이너로 이동
            except Exception as e:
                # 더 이상 해당 인덱스에 genre element가 없으면 종료
                break

        self.book_info['genre'] = genres



        
        # 2. 오버레이 확인 및 제거
        self.check_and_dismiss_overlay(timeout=5)
        
        # 3. 'view more reviews' 버튼 클릭하여 리뷰 페이지로 이동
        try:
            view_more_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, self.view_more_reviews_button_xpath))
            )
            WebDriverWait(self.driver, 10).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.Overlay.Overlay--floating"))
            )
            view_more_button.click()
            time.sleep(2)
        except Exception as e:
            print("리뷰 페이지로 이동하는 'view more reviews' 버튼 클릭 중 오류 발생:", e)
            self.book_info['reviews'] = []
            return self.book_info
        
        # 4. 리뷰 추출 (영어 필터 적용)
        reviews = self.scrap_reviews(target_review_count=self.num_reviews)
        self.book_info['reviews'] = reviews
        
        return self.book_info

    def scrap_reviews(self, target_review_count=100):
        """
        리뷰 컨테이너는
        //*[@id="__next"]/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[INDEX]
        에서 INDEX를 대입하여 접근합니다.
        
        각 컨테이너에서, self.review_xpaths에 저장된 하위 xpath( reviewer, date, rating, review )
        를 사용하여 해당 리뷰의 정보를 추출합니다.
        
        만약 추출에 실패하면(즉, 리뷰 관련 요소가 없으면) 해당 컨테이너를 건너뛰고,
        연속 실패 횟수가 max_failures를 넘으면 "show more reviews" 버튼(//*[@id="__next"]/div[2]/main/div[1]/div[2]/div[5]/div[5]/div/button)
        을 클릭하여 추가 리뷰를 로딩합니다.
        
        target_review_count 만큼 리뷰를 수집하면 리스트를 반환합니다.
        """
        reviews = []
        current_index = 1  # 리뷰 컨테이너 번호 시작 (예: div[1])
        
        max_failures = 3    # 연속 실패 최대 횟수 -> 중간에 광고가 있을 때 건너뛰거나, 2번 실패하면 show more reviews 클릭
        consecutive_failures = 0

        while len(reviews) < target_review_count:
            container_xpath = f"//*[@id='__next']/div[2]/main/div[1]/div[2]/div[5]/div[3]/div[{current_index}]"
            try:
                container = self.driver.find_element(By.XPATH, container_xpath)
                # 컨테이너를 정상적으로 찾으면 실패 카운트를 초기화
                consecutive_failures = 0
            except Exception:
                consecutive_failures += 1
                # 컨테이너 자체를 찾지 못하면, 연속 실패 횟수가 max_failures 이상이면 show more reviews 버튼 클릭 시도
                if consecutive_failures >= max_failures:
                    try:
                        show_more_button = WebDriverWait(self.driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                        )
                        show_more_button.click()
                        time.sleep(2)
                        # 실패 횟수 초기화 후 계속 진행
                        consecutive_failures = 0
                    except Exception as e:
                        print("show more reviews 버튼 클릭 실패:", e)
                        break
                current_index += 1
                continue

            # 각 리뷰 컨테이너에서 하위 요소 추출 시도
            review_data = {}
            extraction_success = True
            for key, xpath_template in self.review_xpaths.items():
                xpath = xpath_template.replace("INDEX", str(current_index))
                try:
                    element = self.driver.find_element(By.XPATH, xpath)
                    if key == "rating":
                        # rating은 aria-label 속성에서 추출
                        review_data[key] = element.get_attribute("aria-label").strip()
                    else:
                        review_data[key] = element.text.strip()
                except Exception:
                    extraction_success = False
                    break
            if extraction_success and review_data.get("review", ""):
                reviews.append(review_data)
                #print(f"수집된 리뷰 {len(reviews)} (컨테이너 div[{current_index}]): {review_data}")
                # 성공 시 실패 횟수 초기화
                consecutive_failures = 0
            else:
                consecutive_failures += 1

            if len(reviews) >= target_review_count:
                break

            current_index += 1

            # 만약 연속 실패 횟수가 max_failures에 도달하면, show more reviews 버튼 클릭 시도
            if consecutive_failures >= max_failures:
                try:
                    show_more_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, self.show_more_button_xpath))
                    )
                    show_more_button.click()
                    time.sleep(2)
                    consecutive_failures = 0
                except Exception as e:
                    print("더 이상 리뷰 로딩 불가 또는 show more reviews 클릭 중 오류 발생:", e)
                    break

        if consecutive_failures >= max_failures:
            print("연속 리뷰 추출 실패 횟수 초과, 리뷰 수집 종료")
        return reviews



    def scrap_list(self):
        """
        1. 책 리스트 페이지에 접속하여, tr[1]부터 tr[num_books]까지의 책 링크( href )를 수집합니다.
        2. 각 책 링크로 이동하여 scrap_book()을 호출해 책 정보와 리뷰를 수집합니다.
        3. 한 페이지의 모든 책을 처리한 후, "next" 버튼(//*[@id="all_votes"]/div[1]/a[12])을 클릭하여
           다음 페이지로 이동하고 같은 과정을 반복합니다.
        4. 최종적으로 모든 책의 데이터를 self.output 에 저장하고 반환합니다.
        """

        current_page = 1
        
        while True:
            print(f"--- 책 리스트 페이지 {current_page} 처리 시작 ---")
            # 현재 페이지 URL로 이동 (페이지 번호가 URL에 반영되어 있지 않다면, next 버튼 클릭으로 변경된 URL를 그대로 사용)
            self.driver.get(self.book_list_url)
            time.sleep(3)
            
            # 한 페이지의 책 링크를 수집 (tr[1] ~ tr[num_books])
            page_links = []
            for i in range(1, 101):
                xpath_link = f"//*[@id='all_votes']/table/tbody/tr[{i}]/td[3]/a"
                try:
                    elem = self.driver.find_element(By.XPATH, xpath_link)
                    link = elem.get_attribute("href")
                    page_links.append(link)
                except Exception as e:
                    print(f"행 {i}의 책 링크 추출 실패: {e}")
                    continue
            
            if not page_links:
                print("현재 페이지에서 책 링크를 하나도 찾지 못했습니다. 종료합니다.")
                break
            
            # 각 책 링크를 순회하며 책 상세 페이지로 이동하고 데이터 수집

            page_output_file = os.path.join(self.output_dir, f"crawler_output_page{current_page}.json")
            book_count = 0
            for link in page_links:
                # href가 상대경로인 경우 기본 도메인 추가

                if link.startswith("/"):
                    link = "https://www.goodreads.com" + link
                print(f"책 링크 처리: {link}")
                self.driver.get(link)
                time.sleep(3)
                self.check_and_dismiss_overlay(timeout=5)
                # scrap_book()은 내부에서 해당 책의 상세 정보와 리뷰를 수집
                book_data = self.scrap_book()
                #all_books.append(book_data)
            
                #output_file = os.path.join(self.output_dir, "crawler_output.json")
                with open(page_output_file, "a", encoding="utf-8") as f:

                    json.dump(book_data, f, ensure_ascii=False, indent=4)
                    f.write("\n")
                print(f"{book_data.get('book_name', 'Unknown Title')} 정보가 저장되었습니다.")
                book_count += 1
                if book_count >= self.num_books:
                    break


            # 한 페이지의 모든 책을 처리한 후, 다음 페이지로 이동
            if self.num_books > 100*current_page:
                try:
                    next_button = WebDriverWait(self.driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//*[@id='all_votes']/div[1]/a[12]"))
                    )
                    next_button.click()
                    time.sleep(3)
                    # 업데이트된 URL(혹은 그대로 사용)이 다음 페이지의 URL
                    self.book_list_url = self.driver.current_url
                    current_page += 1
                except Exception as e:
                    print("다음 페이지 버튼 클릭 실패 또는 더 이상 페이지가 없습니다:", e)
                    break
            else:
                break
        
        #elf.output = all_books
        #return all_books

    def close_browser(self):
        if self.driver:
            self.driver.quit()


# __main__ 코드: 책 리스트에서 첫 번째 책을 선택 후, 책 정보와 리뷰(영어 필터 적용)가 수집됨.
if __name__ == "__main__":
    output_directory = "./database"  # 결과 저장 폴더 (미리 생성)
    path = r"C:\Users\user\.cache\selenium\chromedriver\win64\131.0.6778.264\chromedriver.exe"

    crawler = BookInfoCrawler(output_directory,path, 200, 100)
    crawler.start_browser()
    
    crawler.scrap_list()
    print("전체 loop 수집완료")
    
    crawler.close_browser()


--- 책 리스트 페이지 1 처리 시작 ---
책 링크 처리: https://www.goodreads.com/book/show/2767052-the-hunger-games
로그인 오버레이가 해제되었습니다.
언어 필터 적용 완료
The Hunger Games 정보가 저장되었습니다.
책 링크 처리: https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix
언어 필터 적용 완료
Harry Potter and the Order of the Phoenix 정보가 저장되었습니다.
책 링크 처리: https://www.goodreads.com/book/show/1885.Pride_and_Prejudice
언어 필터 적용 완료
Pride and Prejudice 정보가 저장되었습니다.
책 링크 처리: https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird
언어 필터 적용 완료
To Kill a Mockingbird 정보가 저장되었습니다.
책 링크 처리: https://www.goodreads.com/book/show/19063.The_Book_Thief
언어 필터 적용 완료
The Book Thief 정보가 저장되었습니다.
책 링크 처리: https://www.goodreads.com/book/show/41865.Twilight
언어 필터 적용 완료
Twilight 정보가 저장되었습니다.
책 링크 처리: https://www.goodreads.com/book/show/170448.Animal_Farm
언어 필터 적용 완료
Animal Farm 정보가 저장되었습니다.
책 링크 처리: https://www.goodreads.com/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set
언어 필터 적용 완료
J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of t

In [16]:
crawler.output

[{'book_name': 'The Hunger Games',
  'book_image': 'https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1586722975i/2767052.jpg',
  'author': 'Suzanne Collins',
  'total_star': '4.34',
  'published': 'First published September 14, 2008',
  'pages': 'First published September 14, 2008',
  'description': "Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?\n\nIn the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.\n\nSixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games

In [3]:
import json

# Save crawler.output to a JSON file
with open('crawler_output.json', 'w', encoding='utf-8') as f:
    json.dump(crawler.output, f, ensure_ascii=False, indent=4)